In [ ]:
import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin
from pytenet.fermi_ops import generate_fermi_operators
import numpy as np
from scipy import sparse
import copy
import pytenet as ptn
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt

In [ ]:
#load integrals
with h5py.File("data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)

#load Hamiltonian generated by above coefficients
H_correct = scipy.io.mmread('data_water/H_water_correct.mtx').tocsr()
e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
e_ground = e1

In [ ]:
#generate thc_mpo
t_spin = get_t_spin(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t_spin)

In [ ]:
H_thc = H_mu_nu_list_spin_layer[0][0].as_matrix(sparse_format=True)
H_thc = H_mu_nu_list_spin_layer[0][1].as_matrix(sparse_format=True) @ H_thc
H_thc = H_mu_nu_list_spin_layer[0][2].as_matrix(sparse_format=True) @ H_thc
H_thc = H_mu_nu_list_spin_layer[0][3].as_matrix(sparse_format=True) @ H_thc

#Note: last term in this list is a single MPO for kinetic term
for i in range (1, len(H_mu_nu_list_spin_layer)-1, 1):
    temp = H_mu_nu_list_spin_layer[i][0].as_matrix(sparse_format=True)
    temp = H_mu_nu_list_spin_layer[i][1].as_matrix(sparse_format=True) @ temp
    temp = H_mu_nu_list_spin_layer[i][2].as_matrix(sparse_format=True) @ temp
    temp = H_mu_nu_list_spin_layer[i][3].as_matrix(sparse_format=True) @ temp
    
    H_thc += temp
    
H_thc += H_mu_nu_list_spin_layer[len(H_mu_nu_list_spin_layer)-1][0].as_matrix(sparse_format=True)

print(sparse.linalg.norm(H_thc - H_correct))